In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
x = torch.tensor([[0.0, 0.0],
                  [0.0, 1.0],
                  [1.0, 0.0],
                  [1.0, 1.0]])
y = torch.tensor([[0.0],
                  [1.0],
                  [1.0],
                  [0.0]])

n = x.shape[0]
p = x.shape[1]
d = y.shape[1]
r = 5

class MyModel(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        z1 = self.fc1(x)
        a1 = F.softplus(z1)
        z2 = self.fc2(a1)
        phat = torch.sigmoid(z2)
        return phat

def get_losses(OptClass, nepoch=1000, lr=0.1):
    model = MyModel(in_dim=p, out_dim=d, hidden_dim=r)
    opt = OptClass(model.parameters(), lr=lr)
    losses = []
    
    for i in range(nepoch):
        phat = model(x)
        loss = torch.mean(-y * torch.log(phat) - (1.0 - y) * torch.log(1.0 - phat))

        opt.zero_grad()
        loss.backward()
        opt.step()
        
        losses.append(loss.item())

    return losses

In [3]:
nepoch = 2000
lr = 0.1

np.random.seed(123)
torch.random.manual_seed(123)
losses_sgd = get_losses(torch.optim.SGD, nepoch, lr)

np.random.seed(123)
torch.random.manual_seed(123)
losses_adagrad = get_losses(torch.optim.Adagrad, nepoch, lr)

np.random.seed(123)
torch.random.manual_seed(123)
losses_adadelta = get_losses(torch.optim.Adadelta, nepoch, lr)

np.random.seed(123)
torch.random.manual_seed(123)
losses_rmsprop = get_losses(torch.optim.RMSprop, nepoch, lr)

np.random.seed(123)
torch.random.manual_seed(123)
losses_adam = get_losses(torch.optim.Adam, nepoch, lr)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dat = pd.DataFrame({"x": range(nepoch), "sgd": losses_sgd, "adagrad": losses_adagrad,
                    "adadelta": losses_adadelta, "rmsprop": losses_rmsprop, "adam": losses_adam,
                    "sgd_log": np.log(losses_sgd), "adagrad_log": np.log(losses_adagrad),
                    "adadelta_log": np.log(losses_adadelta), "rmsprop_log": np.log(losses_rmsprop),
                    "adam_log": np.log(losses_adam)})

fig = plt.figure(figsize=(12, 12))
sub = fig.add_subplot(211)
sns.lineplot(data=dat, x="x", y="sgd", label="SGD")
sns.lineplot(data=dat, x="x", y="adagrad", label="Adagrad")
sns.lineplot(data=dat, x="x", y="adadelta", label="Adadelta")
sns.lineplot(data=dat, x="x", y="rmsprop", label="RMSprop")
sns.lineplot(data=dat, x="x", y="adam", label="Adam")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.ylim(0, 1)

sub = fig.add_subplot(212)
sns.lineplot(data=dat, x="x", y="sgd_log", label="SGD")
sns.lineplot(data=dat, x="x", y="adagrad_log", label="Adagrad")
sns.lineplot(data=dat, x="x", y="adadelta_log", label="Adadelta")
sns.lineplot(data=dat, x="x", y="rmsprop_log", label="RMSprop")
sns.lineplot(data=dat, x="x", y="adam_log", label="Adam")
plt.xlabel("Epoch")
plt.ylabel("Log-scale Loss")

dat

ModuleNotFoundError: No module named 'seaborn'